In [1]:
pip install huggingface_hub datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from tqdm.auto import tqdm
import psutil
import os

def get_memory_usage():
    """Get current memory usage in GB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024 / 1024  # Convert bytes to GB

def log_memory(message):
    """Log memory usage with a message"""
    print(f"{message} - Memory usage: {get_memory_usage():.2f} GB")

def process_dataset(dataset, column_name, chunk_size=1000):
    """Process a single dataset in chunks with progress bar"""
    total_len = len(dataset)
    all_images = []
    all_latex = []
    
    # Create chunks of indices
    chunks = range(0, total_len, chunk_size)
    
    # Process chunks with progress bar
    with tqdm(total=total_len, desc="Processing dataset") as pbar:
        for i in chunks:
            # Get chunk indices
            end_idx = min(i + chunk_size, total_len)
            chunk = dataset.select(range(i, end_idx))
            
            # Append data
            all_images.extend(chunk['image'])
            all_latex.extend(chunk[column_name])
            
            # Update progress bar
            pbar.update(end_idx - i)
            
            # Clear memory
            del chunk
            gc.collect()
    
    return all_images, all_latex

def create_splits(images, latex, train_size=0.6, val_size=0.2, test_size=0.2):
    """Create dataset splits with progress bar"""
    total_samples = len(images)
    
    with tqdm(total=3, desc="Creating splits") as pbar:
        # Create train/test split
        train_val_size = train_size + val_size
        train_val_images, test_images, train_val_latex, test_latex = train_test_split(
            images, latex, test_size=test_size, random_state=42
        )
        pbar.update(1)
        
        # Create train/val split
        val_size_adjusted = val_size / train_val_size
        train_images, val_images, train_latex, val_latex = train_test_split(
            train_val_images, train_val_latex, test_size=val_size_adjusted, random_state=42
        )
        pbar.update(1)
        
        # Create datasets
        splits = DatasetDict({
            'train': Dataset.from_dict({'image': train_images, 'latex': train_latex}),
            'validation': Dataset.from_dict({'image': val_images, 'latex': val_latex}),
            'test': Dataset.from_dict({'image': test_images, 'latex': test_latex})
        })
        pbar.update(1)
    
    return splits

def main():
    chunk_size = 20000  # Adjust based on your memory constraints
    
    log_memory("Initial memory usage")
    
    # Load and process first dataset
    print("\nProcessing dataset 1...")
    dataset1 = load_dataset("linxy/LaTeX_OCR", name="synthetic_handwrite", split="train")
    images1, latex1 = process_dataset(dataset1, 'text', chunk_size)
    del dataset1
    gc.collect()
    log_memory("After processing dataset 1")
    
    # Load and process second dataset
    print("\nProcessing dataset 2...")
    dataset2 = load_dataset("OleehyO/latex-formulas", "cleaned_formulas", split="train")
    images2, latex2 = process_dataset(dataset2, 'latex_formula', chunk_size)
    del dataset2
    gc.collect()
    log_memory("After processing dataset 2")
    
    # Combine datasets
    print("\nCombining datasets...")
    all_images = images1 + images2
    all_latex = latex1 + latex2
    del images1, images2, latex1, latex2
    gc.collect()
    log_memory("After combining datasets")
    
    # Create splits
    print("\nCreating and processing splits...")
    splits = create_splits(all_images, all_latex)
    log_memory("After creating splits")
    
    # Push to hub with progress tracking
    print("\nPushing to Hugging Face Hub...")
    splits.push_to_hub("anindya-hf-2002/pix2tex", private=True)
    log_memory("After pushing to hub")

if __name__ == "__main__":
    main()

Initial memory usage - Memory usage: 0.17 GB

Processing dataset 1...


Processing dataset:   0%|          | 0/76266 [00:00<?, ?it/s]

After processing dataset 1 - Memory usage: 10.49 GB

Processing dataset 2...


Processing dataset:   0%|          | 0/552340 [00:00<?, ?it/s]

After processing dataset 2 - Memory usage: 59.33 GB

Combining datasets...
After combining datasets - Memory usage: 59.33 GB

Creating and processing splits...


...

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/629 [00:00<?, ?ba/s]

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/629 [00:00<?, ?ba/s]

After pushing to hub - Memory usage: 64.46 GB


In [3]:
gc.collect()

328

In [1]:
from datasets import load_dataset, Dataset, DatasetDict
import os
from tqdm.auto import tqdm

def process_image_paths(dataset_repo: str, image_dir: str):
    """
    Process a dataset to modify image paths by removing the image_dir prefix
    and keep only relative paths.
    
    Args:
        dataset_repo (str): HuggingFace dataset repository name
        image_dir (str): Base directory path to remove from image paths
    """
    # Normalize image_dir path
    image_dir = os.path.normpath(image_dir)
    if not image_dir.endswith(os.sep):
        image_dir += os.sep
        
    # Load dataset
    print(f"Loading dataset from {dataset_repo}")
    dataset = load_dataset(dataset_repo)
    
    # Process each split
    processed_dataset = {}
    for split_name, split_data in dataset.items():
        print(f"Processing {split_name} split...")
        
        # Get all current paths
        image_paths = split_data['image']
        
        # Process paths to make them relative
        new_paths = []
        for path in tqdm(image_paths, desc=f"Processing {split_name} paths"):
            # Normalize path
            norm_path = os.path.normpath(path)
            # Remove image_dir prefix and convert to forward slashes
            relative_path = norm_path.replace(image_dir, '').replace('\\', '/')
            new_paths.append(relative_path)
        
        # Create new dataset with modified paths
        new_data = {
            'image': new_paths,
            'latex': split_data['latex']
        }
        
        processed_dataset[split_name] = Dataset.from_dict(new_data)
    
    # Create new dataset dictionary
    final_dataset = DatasetDict(processed_dataset)
    
    # Push to HuggingFace
    try:
        print(f"Pushing processed dataset to anindya-hf-2002/pix2tex_data")
        final_dataset.push_to_hub(f"anindya-hf-2002/pix2tex_data")
        print("Successfully pushed to HuggingFace!")
    except Exception as e:
        print(f"Error pushing to hub: {e}")
        # Save locally as fallback
        final_dataset.save_to_disk("processed_dataset")
        print("Dataset saved locally as 'processed_dataset'")
    
    return final_dataset

# Example usage
if __name__ == "__main__":
    DATASET_REPO = "anindya-hf-2002/pix2tex_processed"
    IMAGE_DIR = "/teamspace/studios/this_studio/Pix2Tex/data"
    
    try:
        dataset = process_image_paths(DATASET_REPO, IMAGE_DIR)
        
        # Verify a few examples
        for split_name, split_data in dataset.items():
            print(f"\nExample paths from {split_name} split:")
            for i in range(min(3, len(split_data))):
                print(split_data[i]['image'])
    except Exception as e:
        print(f"Error during processing: {e}")

Loading dataset from anindya-hf-2002/pix2tex_processed


README.md:   0%|          | 0.00/537 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/34.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/126 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/126 [00:00<?, ?ba/s]

Successfully pushed to HuggingFace!

Example paths from train split:
train/img_00000000.png
train/img_00000001.png
train/img_00000002.png

Example paths from validation split:
validation/img_00000000.png
validation/img_00000001.png
validation/img_00000002.png

Example paths from test split:
test/img_00000000.png
test/img_00000001.png
test/img_00000002.png
